# Advanced tour of the Bayesian Optimization package

In [1]:
from bayes_opt import BayesianOptimization

## 1. Suggest-Evaluate-Register Paradigm

Internally the `maximize` method is simply a wrapper around the methods `suggest`, `probe`, and `register`. If you need more control over your optimization loops the Suggest-Evaluate-Register paradigm should give you that extra flexibility.

For an example of running the `BayesianOptimization` in a distributed fashion (where the function being optimized is evaluated concurrently in different cores/machines/servers), checkout the `async_optimization.py` script in the examples folder.

In [2]:
# Let's start by defining our function, bounds, and instantiating an optimization object.
def black_box_function(x, y):
    return -x ** 2 - (y - 1) ** 2 + 1

One extra ingredient we will need is an `AcquisitionFunction`, such as `acquisition.UpperConfidenceBound`. In case it is not clear why, take a look at the literature to understand better how this method works.

In [3]:
from bayes_opt import acquisition

acq = acquisition.UpperConfidenceBound(kappa=2.5)

Notice that the evaluation of the blackbox function will NOT be carried out by the optimizer object. We are simulating a situation where this function could be being executed in a different machine, maybe it is written in another language, or it could even be the result of a chemistry experiment. Whatever the case may be, you can take charge of it and as long as you don't invoke the `probe` or `maximize` methods directly, the optimizer object will ignore the blackbox function.

In [4]:
optimizer = BayesianOptimization(
    f=None,
    acquisition_function=acq,
    pbounds={'x': (-2, 2), 'y': (-3, 3)},
    verbose=2,
    random_state=1,
)

The `suggest` method of our optimizer can be called at any time. What you get back is a suggestion for the next parameter combination the optimizer wants to probe.

Notice that while the optimizer hasn't observed any points, the suggestions will be random. However, they will stop being random and improve in quality the more points are observed.

In [5]:
next_point_to_probe = optimizer.suggest()
print("Next point to probe is:", next_point_to_probe)

Next point to probe is: {'x': np.float64(-0.331911981189704), 'y': np.float64(1.3219469606529486)}


You are now free to evaluate your function at the suggested point however/whenever you like.

In [6]:
target = black_box_function(**next_point_to_probe)
print("Found the target value to be:", target)

Found the target value to be: 0.7861845912690544


Last thing left to do is to tell the optimizer what target value was observed.

In [7]:
optimizer.register(
    params=next_point_to_probe,
    target=target,
)

### 1.1 The maximize loop

And that's it. By repeating the steps above you recreate the internals of the `maximize` method. This should give you all the flexibility you need to log progress, hault execution, perform concurrent evaluations, etc.

In [8]:
for _ in range(5):
    next_point = optimizer.suggest()
    target = black_box_function(**next_point)
    optimizer.register(params=next_point, target=target)
    
    print(target, next_point)
print(optimizer.max)

-18.707136686093495 {'x': np.float64(1.9261486197444082), 'y': np.float64(-2.9996360060323246)}
0.750594563473972 {'x': np.float64(-0.3763326769822668), 'y': np.float64(1.328297354179696)}
-6.559031075654336 {'x': np.float64(1.979183535803597), 'y': np.float64(2.9083667381450318)}
-6.915481333972961 {'x': np.float64(-1.9686133847781613), 'y': np.float64(-1.009985740060171)}
-6.8600832617014085 {'x': np.float64(-1.9763198875239296), 'y': np.float64(2.9885278383464513)}
{'target': np.float64(0.7861845912690544), 'params': {'x': np.float64(-0.331911981189704), 'y': np.float64(1.3219469606529486)}}


## 2. Tuning the underlying Gaussian Process

The bayesian optimization algorithm works by performing a gaussian process regression of the observed combination of parameters and their associated target values. The predicted parameter $\rightarrow$ target hyper-surface (and its uncertainty) is then used to guide the next best point to probe.

### 2.1 Passing parameter to the GP

Depending on the problem it could be beneficial to change the default parameters of the underlying GP. You can use the `optimizer.set_gp_params` method to do this:

In [9]:
optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds={'x': (-2, 2), 'y': (-3, 3)},
    verbose=2,
    random_state=1,
)
optimizer.set_gp_params(alpha=1e-3, n_restarts_optimizer=5)
optimizer.maximize(
    init_points=1,
    n_iter=5
)

|   iter    |  target   |     x     |     y     |
-------------------------------------------------
| 1         | 0.7862    | -0.331911 | 1.3219469 |
| 2         | -18.34    | 1.9021640 | -2.965222 |
| 3         | 0.8731    | -0.298167 | 1.1948749 |
| 4         | -6.497    | 1.9876938 | 2.8830942 |
| 5         | -4.286    | -1.995643 | -0.141769 |
| 6         | -6.781    | -1.953302 | 2.9913127 |


### 2.2 Tuning the `alpha` parameter

When dealing with functions with discrete parameters,or particularly erratic target space it might be beneficial to increase the value of the `alpha` parameter. This parameters controls how much noise the GP can handle, so increase it whenever you think that extra flexibility is needed.

### 2.3 Changing kernels

By default this package uses the Matern 2.5 kernel. Depending on your use case you may find that tuning the GP kernel could be beneficial. You're on your own here since these are very specific solutions to very specific problems. You should start with the [scikit learn docs](https://scikit-learn.org/stable/modules/gaussian_process.html#kernels-for-gaussian-processes).

## Observers Continued

Observers are objects that subscribe and listen to particular events fired by the `BayesianOptimization` object. 

When an event gets fired a callback function is called with the event and the `BayesianOptimization` instance passed as parameters. The callback can be specified at the time of subscription. If none is given it will look for an `update` method from the observer.

In [10]:
from bayes_opt.event import DEFAULT_EVENTS, Events

In [11]:
optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds={'x': (-2, 2), 'y': (-3, 3)},
    verbose=2,
    random_state=1,
)

In [12]:
class BasicObserver:
    def update(self, event, instance):
        """Does whatever you want with the event and `BayesianOptimization` instance."""
        print("Event `{}` was observed".format(event))

In [13]:
my_observer = BasicObserver()

optimizer.subscribe(
    event=Events.OPTIMIZATION_STEP,
    subscriber=my_observer,
    callback=None, # Will use the `update` method as callback
)

Alternatively you have the option to pass a completely different callback.

In [14]:
def my_callback(event, instance):
    print("Go nuts here!")

optimizer.subscribe(
    event=Events.OPTIMIZATION_START,
    subscriber="Any hashable object",
    callback=my_callback,
)

In [15]:
optimizer.maximize(init_points=1, n_iter=2)

Go nuts here!
Event `optimization:step` was observed
Event `optimization:step` was observed
Event `optimization:step` was observed


For a list of all default events you can checkout `DEFAULT_EVENTS`

In [16]:
DEFAULT_EVENTS

['optimization:start', 'optimization:step', 'optimization:end']